## Problem Statement/Goal
 - 

## Imports

In [313]:
# Standard Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Scraping Libraries
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from time import sleep
import re

# NLP Libraries
from textblob import TextBlob
import textstat

def Sentamentize(text):
    return TextBlob(str(text)).sentiment.polarity

In [562]:
def open_browser(alt_user_name = 'Thank you for your website'):
    opts = Options()
    opts.add_argument("user-agent=" + str(alt_user_name))
    path = '../Garage/chromedriver'                                         # Path to Chromedriver
    return webdriver.Chrome(executable_path = path, options=opts)#, options=ops)       # Open browser

# Original/First Document

### Initial Viewing of raw data

In [667]:
s_list = full_doc.split("""......................................................................................................................................................................................................................................................""")
for i in s_list[6:9]:
    print('------')
    print(i.split('\n'))

------
['', 'Nancy Paxton', 'Senior Director, Investor Relations and Treasury, Apple, Inc.', "Thank you, Luca, and we'd like to ask that you limit yourself to two questions. Operator, may we have the first question please?", '']
------
['', 'QUESTION AND ANSWER SECTION', 'Operator: Certainly. Our first question will come from Wamsi Mohan with Bank of America Merrill Lynch. ']
------
['', 'Wamsi Mohan Q Analyst, Bank of America Merrill Lynch', 'Yes. Thank you. Tim, there has been some real deceleration in some of these emerging markets, partly driven by', 'some concerns around some of the rules the administration is contemplating and partly driven by things that are', 'more specific to China, for instance, like some of the regulations around gaming. So can you talk about how you', 'see the trajectory there for the business and what you think of the initiatives of some companies like Netflix and', 'Fortnite trying to bypass the App Store around subscriptions and I have a follow-up.', '']

### Cleaning data

In [668]:
# Manually removing the things that were cluttering things but not easily removed.
clean_list = ['',' ','  ','   ','    ','A','1 ', '2 ', '3 ','4 ','5 ','6 ','7 ',
              '8 ','9 ','10 ','11 ','12 ','13 ','14 ','15 ','16 ',
              'Apple, Inc. (AAPL) Q4 2018 Earnings Call',
              '2 1-877-FACTSET www.callstreet.com Copyright  2001-2018 FactSet CallStreet, LLC',
              '10 1-877-FACTSET www.callstreet.com Copyright  2001-2018 FactSet CallStreet, LLC',
              '12 1-877-FACTSET www.callstreet.com Copyright  2001-2018 FactSet CallStreet, LLC',
              '14 1-877-FACTSET www.callstreet.com Copyright  2001-2018 FactSet CallStreet, LLC',
              '16 1-877-FACTSET www.callstreet.com Copyright  2001-2018 FactSet CallStreet, LLC',
              '17 1-877-FACTSET www.callstreet.com Copyright  2001-2018 FactSet CallStreet, LLC',
              'Apple, Inc. (AAPL) Corrected Transcript Q4 2018 Earnings Call 01-Nov-2018']

In [669]:
# Creating a DataFrame from a list of lists.
splits = [mini_list.replace('©','').split('\n') for mini_list in s_list]
df = pd.DataFrame([[item for item in mini_list if not item in clean_list] for mini_list in splits])

# Dropping specific rows that aren't what we're looking for but were created anyways
df.drop([0,1,2,7,15,22,28,36,42], inplace=True)

# Removing specific cells that were better off empty
for i in [3,4,5,6,32,41]:
    df[1][i] = None

In [670]:
df['full_statement'] = df[df.columns[1:]].apply(lambda x: ','.join(x.dropna()), axis=1)

df.drop([i for i in df.columns][1:-1], # list of all columns that aren't the 0 or full_statement
        axis=1, inplace = True)

In [671]:
df.head(3)

,0,full_statement
3,Nancy Paxton,Thank you. Good afternoon and thanks to everyo...
4,Timothy Donald Cook,"Thank you, Nancy. Good afternoon everyone, and..."
5,Luca Maestri,"Thank you, Tim. Good afternoon everyone.,5 1-8..."


### Obtaining Desired Data

In [672]:
# Creating a sentiment column from the full_statement column
df['sentiment'] = df['full_statement'].map(lambda x: Sentamentize(x))

# Creating a complexity column from the full_statement column
df['GF_complexity'] = df['full_statement'].map(lambda x: textstat.gunning_fog(x))

df.head()
# Still looking good

,0,full_statement,sentiment,GF_complexity
3,Nancy Paxton,Thank you. Good afternoon and thanks to everyo...,0.155357,23.48
4,Timothy Donald Cook,"Thank you, Nancy. Good afternoon everyone, and...",0.298237,21.14
5,Luca Maestri,"Thank you, Tim. Good afternoon everyone.,5 1-8...",0.175295,19.92
6,Nancy Paxton,"Thank you, Luca, and we'd like to ask that you...",0.250000,11.56
8,"Wamsi Mohan Q Analyst, Bank of America Merrill...","Yes. Thank you. Tim, there has been some real ...",0.100000,25.68


In [673]:
# Saving the data
df.to_csv('../NLP_of_Earning_Calls.csv')

# Larger Project

In [509]:
### This was an attempt to have all of the transcripts added at one time 
# with all of the splitting and sentiment done at once.  While this wasn't
# a bad idea, having so man pieces meant that it made everything more 
# difficult to fix if something broke.  This is why I later broke everything 
# up and took it one step at a time.
browser         = open_browser()
num_transcripts = 30
list_of_dicts   = []
for page_num in range(1,7):
    current_ts_list = 'https://seekingalpha.com/earnings/earnings-call-transcripts/' + str(page_num)
    for ts_num in range(1,num_transcripts):
        browser.get(current_ts_list)                                            # On forum page
        list_elements           = browser.find_elements_by_class_name('dashboard-article-link')
        num_transcripts_on_page = len(list_elements)
        current_ts              = list_elements[ts_num]                         # On individual transcript   
        current_ts.click()
        browser.get(browser.current_url + '?part=single')
        soup                    = BeautifulSoup(browser.page_source)            # 
        article                 = soup.find('article')
        p_elements              = [item.text for item in article.find_all('p')]
        title                   = p_elements[0]
        print('scraping ',title)
        sleep()
        
        # Finding the seperation between main speech(es) and QA section
        done = False
        for item_num in range(len(p_elements)):
            if done == True:
                break
            elif p_elements[item_num] == 'Question-and-Answer Session':
                pree_QA_title         = p_elements[:item_num - 1]
                post_QA_title         = p_elements[item_num:]
                done                  = True
            else:
                pass
            
        speech = ''.join([i for i in pree_QA_title if len(i) >= 35][1:])
        QA     = ''.join([i for i in post_QA_title if len(i) >= 25])
        current_dict = {
            'title'   : test_title,
            'speech'  : speech,
            'Q_and_A' : QA,
            'speech_sentiment' : Sentamentize(speech),
            'speech_complexity': textstat.gunning_fog(speech),
            'QA_sentiment'     : Sentamentize(QA),
            'QA_complexity'    : textstat.gunning_fog(QA)
        }
        list_of_dicts.append(current_dict)
        
        
df = pd.DataFrame(list_of_dicts, columns = ['title', 'speech', 'Q_and_A',
                                            'speech_sentiment','speech_complexity',
                                            'QA_sentiment','QA_complexity'])

scraping  Klepierre SA (OTCPK:KLPEF) Q4 2018 Earnings Conference Call February  7, 2019  3:00 AM ET
scraping  ESCO Technologies Inc. (NYSE:ESE) Q1 2019 Earnings Conference Call February  7, 2019  5:00 PM ET
scraping  Electrovaya Inc. (OTCQX:EFLVF) Q1 2019 Results Earnings Conference Call February  7, 2019  7:30 AM ET
scraping  Great-West Lifeco’s. (OTCPK:GWLIF) Q4 2018 Results Earnings Conference Call February  7, 2019  3:30 PM ET
scraping  Ultralife Corp. (NASDAQ:ULBI) Q4 2018 Earnings Conference Call February  7, 2019 10:00 AM ET
scraping  Cleveland-Cliffs Inc. (NYSE:CLF) Q4 2018 Earnings Conference Call February  8, 2019  9:00 AM ET
scraping  Kemira Oyj (OTC:KMRAF) Q4 2018 Earnings Conference Call February  8, 2019  3:30 AM ET
scraping  Matrix Service Company (NASDAQ:MTRX) Q2 2019 Earnings Conference Call February  7, 2019 10:30 AM ET


AttributeError: 'NoneType' object has no attribute 'find_all'

## Data Collection

In [572]:
# Final Version for scraping the the urls of each transcript
for page_num in range(140,1001):
    browser = open_browser("Y'all are doing a great job " + str(page_num))
    current_ts_list = 'https://seekingalpha.com/earnings/earnings-call-transcripts/' + str(page_num)
    browser.get(current_ts_list)
    elements_list = browser.find_elements_by_class_name('dashboard-article-link')
    urls    = [el.get_attribute('href') + '?part=single' for el in elements_list]
    headers = [el.text for el in elements_list]
    print('scraping', page_num)
    #sleep(20)
    for transcript_num in range(len(urls)):
        current_transcript = {
            'header'   : headers[transcript_num],
            'url'      : urls[transcript_num],
            'page_list': page_num
        }
        url_list.append(current_transcript)
    browser.close()

scraping 140
scraping 141
scraping 142
scraping 143
scraping 144
scraping 145
scraping 146
scraping 147
scraping 148
scraping 149
scraping 150
scraping 151
scraping 152
scraping 153
scraping 154
scraping 155
scraping 156
scraping 157
scraping 158
scraping 159
scraping 160
scraping 161
scraping 162
scraping 163
scraping 164
scraping 165
scraping 166
scraping 167
scraping 168
scraping 169
scraping 170
scraping 171
scraping 172
scraping 173
scraping 174
scraping 175
scraping 176
scraping 177
scraping 178
scraping 179
scraping 180
scraping 181
scraping 182
scraping 183
scraping 184
scraping 185
scraping 186
scraping 187
scraping 188
scraping 189
scraping 190
scraping 191
scraping 192
scraping 193
scraping 194
scraping 195
scraping 196
scraping 197
scraping 198
scraping 199
scraping 200
scraping 201
scraping 202
scraping 203
scraping 204
scraping 205
scraping 206
scraping 207
scraping 208
scraping 209
scraping 210
scraping 211
scraping 212
scraping 213
scraping 214
scraping 215
scraping 216

KeyboardInterrupt: 

In [713]:
for i in range(20):
    try:
        for row_num in range(len(url_df['transcript'])):
            if url_df['transcript'][row_num] == 0:
                # Go to url and obtain the full transcript
                browser = open_browser()#str(random.randrange(0,1000)) + "Style & Grace")
                browser.get(url_df['url'][row_num])
                print('scraping',row_num,
                      datetime.fromtimestamp(time()).strftime('%H:%M:%S'),
                      url_df['header'][row_num][:85])
                
                soup        = BeautifulSoup(browser.page_source)
                article     = soup.find('article')
                url_df['transcript'][row_num]  = [item.text for item in article.find_all('p')]
                browser.close()
                sleep(20)
            else:
                pass
    except:
        print('--------')
        print('ERROR', i)
        print('--------')
        sleep(100)
        browser.close()

scraping 4010 19:49:26 Select Medical Holdings Corporation (SEM) Q3 2018 Results - Earnings Call Transcript


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


scraping 4011 19:50:16 STAG Industrial, Inc. (STAG) CEO Ben Butcher on Q3 2018 Results - Earnings Call Trans
scraping 4012 19:50:48 Casella Waste Systems, Inc. (CWST) CEO John Casella on Q3 2018 Results - Earnings Cal
scraping 4013 19:51:22 Arbor Realty Trust, Inc. (ABR) CEO Ivan Kaufman on Q3 2018 Results - Earnings Call Tr
scraping 4014 19:51:55 Triton International Limited (TRTN) CEO Brian Sondey on Q3 2018 Results - Earnings Ca
scraping 4015 19:52:27 Fairfax Financial Holdings Ltd. (FRFHF) Q3 2018 Results - Earnings Call Transcript
scraping 4016 19:52:55 Exxon Mobil (XOM) Q3 2018 Results - Earnings Call Transcript
scraping 4017 19:53:33 W.P. Carey, Inc. (WPC) CEO Jason Fox on Q3 2018 Results - Earnings Call Transcript
scraping 4018 19:54:09 AptarGroup's (ATR) CEO Stephan Tanda on Q3 2018 Results - Earnings Call Transcript
scraping 4019 19:54:42 Monotype Imaging Holdings' (TYPE) CEO Scott Landers on Q3 2018 Results - Earnings Cal
scraping 4020 19:55:17 Hydrogenics Corp (HYGS) CEO Da

scraping 4090 20:42:13 ArcelorMittal SA (MT) Q3 2018 Results - Earnings Call Transcript
scraping 4091 20:42:49 Archrock, Inc. (AROC) CEO Brad Childers on Q3 2018 Results - Earnings Call Transcript
scraping 4092 20:43:23 The Western Union (WU) Q3 2018 Results - Earnings Call Transcript
scraping 4093 20:43:58 Puma Biotechnology (PBYI) Q3 2018 Results - Earnings Call Transcript
scraping 4094 20:44:25 Weight Watchers International (WTW) Q3 2018 Results - Earnings Call Transcript
scraping 4095 20:45:03 Arista Networks (ANET) Q3 2018 Results - Earnings Call Transcript
scraping 4096 20:45:37 Canadian Natural Resources, Ltd. (CNQ) Q3 2018 Results - Earnings Call Transcript
scraping 4097 20:46:09 The Kraft Heinz (KHC) Q3 2018 Results - Earnings Call Transcript
scraping 4098 20:46:45 Hercules Capital, Inc. (HTGC) CEO Manuel Henriquez on Q3 2018 Results - Earnings Call
scraping 4099 20:47:19 Shake Shack (SHAK) CEO Randy Garutti on Q3 2018 Results - Earnings Call Transcript
scraping 4100 20:47:54 

scraping 4171 21:33:59 Cirrus Logic (CRUS) Q2 2019 Results - Earnings Call Transcript
scraping 4172 21:34:38 Apache (APA) Q3 2018 Results - Earnings Call Transcript
scraping 4173 21:35:09 CorEnergy Infrastructure Trust Inc. (CORR) CEO David Schulte on Q3 2018 Results - Ear
scraping 4174 21:35:42 ViaSat (VSAT) Q2 2019 Results - Earnings Call Transcript
scraping 4175 21:36:14 LifeVantage Corporation (LFVN) CEO Darren Jensen on Q1 2019 Results - Earnings Call T
scraping 4176 21:36:46 Career Education Corp (CECO) CEO Todd Nelson on Q3 2018 Results - Earnings Call Trans
scraping 4177 21:37:19 The Allstate (ALL) Q3 2018 Results - Earnings Call Transcript
scraping 4178 21:37:52 AAON, Inc. (AAON) Q3 2018 Results - Earnings Call Transcript
scraping 4179 21:38:24 Cerus Corporation (CERS) CEO Obi Greenman on Q3 2018 Results - Earnings Call Transcri
scraping 4180 21:38:56 Vivus Inc. (VVUS) CEO John Amos on Q3 2018 Results - Earnings Call Transcript
scraping 4181 21:39:28 NewLink Genetics Corp (NLN

...

scraping 5768 14:40:24 VOXX International Corp (VOXX) CEO Pat Lavelle on Q2 2019 Results - Earnings Call Tra
scraping 5769 14:40:59 National American University Holdings, Inc. (NAUH) CEO Ronald Shape on Q1 2019 Result
scraping 5770 14:41:37 Nightfood Holdings, Inc. (NGTF) CEO Sean Folkson Hosts Investor Conference Call (Tran
scraping 5771 14:42:15 LVMH-Moet Hennessy Louis Vuitton (LVMHF) Q3 2018 Results - Earnings Call Transcript
scraping 5772 14:42:51 Fastenal Company (FAST) CEO Dan Florness on Q3 2018 Results - Earnings Call Transcrip
scraping 5773 14:43:29 DSW Inc. (DSW) CEO Roger Rawlins Hosts DSW and Authentic Brands Group Announce Acquis
scraping 5774 14:44:19 BlackPearl Resources Inc. (BLKPF) International Petroleum Corporation to Acquire Blac
scraping 5775 14:45:03 MTY Food Group Inc. (MTYFF) Q3 2018 Results - Earnings Call Transcript
scraping 5776 14:45:41 IDT Corporation (IDT) CEO Shmuel Jonas on Q4 2018 Results - Earnings Call Transcript
scraping 5777 14:46:22 AZZ incorporat

scraping 5845 15:29:22 Boohoo Group Plc ADR (BHOOY) Management on Q2 2019 Results - Earnings Call Transcript
scraping 5846 15:30:01 Roche Holding Ltd (RHHBY) Management on Virtual Pipeline Event 2018 World Conference 
scraping 5847 15:30:38 Alexion Pharmaceuticals, Inc. (ALXN) CEO Ludwig Hantson on Acquisition of Syntimmune 
scraping 5848 15:31:16 AGF Management Ltd. (AGFMF) CEO Blake Goldring on Q3 2018 Results - Earnings Call Tra
scraping 5849 15:31:50 Worthington Industries, Inc. (WOR) CEO John McConnell on Q1 2019 Results - Earnings C
scraping 5850 15:32:29 OMNOVA Solutions Inc. (OMN) CEO Anne Noonan on Q3 2018 Results - Earnings Call Transc
scraping 5851 15:33:06 Electromed, Inc. (ELMD) CEO Kathleen Skarvan on Q4 2018 Results - Earnings Call Trans
scraping 5852 15:33:45 CarMax Group (KMX) CEO, Bill Nash on Q2 2019 Results - Earnings Call Transcript
scraping 5853 15:34:23 NetSol's (NTWK) CEO Najeeb Ghauri on Q4 2018 Results - Earnings Call Transcript
scraping 5854 15:35:26 CLPS, In

scraping 5922 16:30:21 Adobe Systems, Inc. (ADBE) CEO Shantanu Narayen on Q3 2018 Results - Earnings Call Tr
scraping 5923 16:31:03 China Finance Online Co. Limited (JRJC) CEO Zhiwei Zhao on Q2 2018 Results - Earnings
scraping 5924 16:31:43 Lithia Motors' (LAD) CEO Bryan DeBoer Announces Strategic Partnership with Shift Tech
scraping 5925 16:32:23 Park City Group, Inc. (PCYG) CEO Randy Fields on Q4 2018 Results - Earnings Call Tran
--------
ERROR 12
--------
scraping 5926 16:39:47 Frequency Electronics, Inc. (FEIM) CEO Stanton Sloane on Q1 2019 Results - Earnings C
scraping 5927 16:40:31 Nevada Gold & Casinos, Inc (UWN) CEO Mike Shaunnessy on Q1 2019 Results - Earnings Ca
scraping 5928 16:41:11 JPMorgan Chase & Co. (JPM) Management at the Barclays Global Financial Services Confe
scraping 5929 16:41:50 Cherokee Inc. (CHKE) CEO Henry Stupp on Q2 2019 Results - Earnings Call Transcript
scraping 5930 16:42:38 NXP Semiconductors NV (NXPI) Presents at Deutsche Bank 2018 Technology Conference

scraping 5998 17:37:30 Hudson's Bay Co. (HBAYF) CEO Helena Foulkes on HBC Enters Agreement to Merge HBC Euro
scraping 5999 17:38:14 Cairn Energy Plc (CRNCY) CEO Simon Thomson on Q2 2018 Results - Earnings Call Transcr
scraping 6000 17:38:56 Francesca's Holdings Corp. (FRAN) CEO Steve Lawrence on Q2 2018 Results - Earnings Ca
scraping 6001 17:39:35 Optical Cable Corporation (OCC) CEO Neil Wilkin on Q3 2018 Results - Earnings Call Tr
--------
ERROR 14
--------


KeyboardInterrupt: 

## Viewing of Raw Data

In [719]:
df = url_df.head(6000)

In [722]:
df.drop(['page_list', 'url'], axis = 1, inplace = True)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


## Cleaning

## Exploritory Data Analysis

## Modeling

### Modeling Imports & Initiations

### Historical Lists of Features & Their Results

# Laboritory!
 - The place for all the code that I'm currently working on.  
 - This is to keep the rest of the notebook clean.

## Previous versions of Code

In [485]:
speech  = ''.join(pree_QA_title)
QA      = ''.join(post_QA_title)
test_list_of_dicts = [
{
    'title'   : test_title,
    'speech'  : speech,
    'Q_and_A' : QA,
    'speech_sentiment' : Sentamentize(speech),
    'speech_complexity': textstat.gunning_fog(speech),
    'QA_sentiment'     : Sentamentize(QA),
    'QA_complexity'    : textstat.gunning_fog(QA)
}]

In [672]:
# Creating a sentiment column from the full_statement column
df['sentiment'] = df['full_statement'].map(lambda x: Sentamentize(x))

# Creating a complexity column from the full_statement column
df['GF_complexity'] = df['full_statement'].map(lambda x: textstat.gunning_fog(x))

df.head()
# Still looking good

,0,full_statement,sentiment,GF_complexity
3,Nancy Paxton,Thank you. Good afternoon and thanks to everyo...,0.155357,23.48
4,Timothy Donald Cook,"Thank you, Nancy. Good afternoon everyone, and...",0.298237,21.14
5,Luca Maestri,"Thank you, Tim. Good afternoon everyone.,5 1-8...",0.175295,19.92
6,Nancy Paxton,"Thank you, Luca, and we'd like to ask that you...",0.250000,11.56
8,"Wamsi Mohan Q Analyst, Bank of America Merrill...","Yes. Thank you. Tim, there has been some real ...",0.100000,25.68


# Analyzing the Current Data

In [734]:
QAS = 'Question-and-Answer Session'
df['transcript'][1][df['transcript'][1].index(QAS):]

['Question-and-Answer Session',
 'Q - Michel Varaldo',
 'Michel Varaldo, Société Générale. Two question for me. So first one it is retailer sales. We have a trend, which is still positive but declining year-by-year. And we have the negative figures in Italy, in Scandinavian countries. Is it because we have a [indiscernible] kind of cannibalization from the sales in your shopping centers through internet sales? Is it possible to have a little more color about this? And the second question is about the investment market. We have a lower activity last year, how is it in the beginning of this year in term of acquisition, disposal? Will you continue to sell at the same level for this year, for example? Thank you.',
 'Jean-Marc Jestin',
 'Thank you, Michel. So, the retailer sales, I don’t have any crystal ball to look in the future, but the way we look at it is, first of all, I we are active in 12 countries, and then the as you know, the macro economies are never synchronized. So, and the mo

In [762]:
df.head()

,header,speech,QA,transcript
0,Vocera Communications (VCRA) CEO Brent Lang on...,,"[Question-and-Answer Session, Operator, [Opera...",[Vocera Communications (NYSE:VCRA) Q4 2018 Ear...
1,Klepierre SA (KLPEF) CEO Jean-Michel Rene Gaul...,Jean-Michel Rene Gault - Deputy Chief Executiv...,"[Question-and-Answer Session, Q - Michel Varal...",[Klepierre SA (OTCPK:KLPEF) Q4 2018 Earnings C...
2,ESCO Technologies Inc. (ESE) CEO Vic Richey on...,Vic Richey - Chairman and Chief Executive Offi...,"[Question-and-Answer Session, Operator, Thank ...",[ESCO Technologies Inc. (NYSE:ESE) Q1 2019 Ear...
3,Electrovaya Inc. (EFLVF) Management on Q1 2019...,Richard Halka - Executive Vice President and C...,"[Question-and-Answer Session, Operator, Raj Da...",[Electrovaya Inc. (OTCQX:EFLVF) Q1 2019 Result...
4,Great-West Lifeco's (GWLIF) CEO Paul Mahon on ...,Garry MacNicholas - Executive Vice President a...,"[Question-and-Answer Session, Operator, Certai...",[Great-West Lifeco’s. (OTCPK:GWLIF) Q4 2018 Re...


In [772]:
def speech_function(x):
    if QAS in x:
        result = ''.join([statement for statement in x[1:x.index(QAS)]if len(statement) >= 100])
    else:
        result = np.nan
    return result
    
def QA_sec_function(x):
    if QAS in x:
        result = ''.join([statement for statement in x[x.index(QAS) +1:] if len(statement) >= 25])
    else:
        result = np.nan
    return result

df['speech'] = df['transcript'].map(lambda x: speech_function(x))
df['QA']     = df['transcript'].map(lambda x: QA_sec_function(x))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [777]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 6 columns):
header              6000 non-null object
speech              5829 non-null object
QA                  5829 non-null object
transcript          6000 non-null object
speech_sentiment    6000 non-null float64
QA_sentiment        6000 non-null float64
dtypes: float64(2), object(4)
memory usage: 281.3+ KB


In [778]:
df['speech'].fillna('', inplace = True)
df['QA'].fillna('', inplace = True)
print("na's filled")

# Creating a sentiment columns from the speech & QA columns
df['speech_sentiment']  = df['speech'].map(lambda x: Sentamentize(x))
print('speech_sentiment Done')
df['QA_sentiment']      = df['QA'].map(lambda x: Sentamentize(x))
print('QA_sentiment Done')

# Creating a complexity column from the speech & QA columns
df['speech_complexity'] = df['speech'].map(lambda x: textstat.gunning_fog(x))
print('speech_complexity Done')
df['QA_complexity']     = df['QA'].map(lambda x: textstat.gunning_fog(x))
print('QA_complexity Done')


/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


na's filled


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


speech_sentiment Done


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


QA_sentiment Done


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


speech_complexity Done
QA_complexity Done


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [781]:
df.to_csv('./../EC_Analyzed.csv', index = False)

In [771]:
print(speech_function(df['transcript'][0]))
df.head()

['Good afternoon, ladies and gentlemen and welcome to the Vocera Communications Conference Call. My name is Chris and I will be your coordinator for today. [Operator Instructions] Thank you. I would now like to turn the presentation over to your host for today’s call, Sue Dooley, Vocera’s Director of Investor Relations. Please proceed.', 'Hello, everyone. Welcome to Vocera’s conference call to discuss our fourth quarter fiscal 2018 earnings. This is Sue Dooley. And joining me today are Vocera’s CEO, Brent Lang and our CFO, Justin Spencer.', 'We distributed a press release detailing our quarterly results earlier this afternoon. The release is posted on our website at investors.vocera.com and is also available from normal news sources. This conference call is being webcast live on the IR page of our website where a replay will be archived.', 'Before we begin our prepared remarks, I would like to take this opportunity to remind you that during the course of this call, we will make forward

,header,speech,QA,transcript
0,Vocera Communications (VCRA) CEO Brent Lang on...,"[Good afternoon, ladies and gentlemen and welc...",[[Operator Instructions] Your first question i...,[Vocera Communications (NYSE:VCRA) Q4 2018 Ear...
1,Klepierre SA (KLPEF) CEO Jean-Michel Rene Gaul...,"[Good morning, everyone. Jean-Michel and mysel...","[Michel Varaldo, Société Générale. Two questio...",[Klepierre SA (OTCPK:KLPEF) Q4 2018 Earnings C...
2,ESCO Technologies Inc. (ESE) CEO Vic Richey on...,[Good day and welcome to the ESCO Technologies...,[Thank you. [Operator Instructions] And our fi...,[ESCO Technologies Inc. (NYSE:ESE) Q1 2019 Ear...
3,Electrovaya Inc. (EFLVF) Management on Q1 2019...,"[Greetings, and welcome to the Electrovaya’s C...",[Yes. That concludes our call. Thank you for l...,[Electrovaya Inc. (OTCQX:EFLVF) Q1 2019 Result...
4,Great-West Lifeco's (GWLIF) CEO Paul Mahon on ...,"[Good afternoon, and welcome to Great-West Lif...","[Certainly, sir. [Operator Instructions] And t...",[Great-West Lifeco’s. (OTCPK:GWLIF) Q4 2018 Re...


In [682]:
count = 0
for trans in df['transcript']:                                  # For each item in the transcripts column.
    done = False
    for trans_num in range(len(trans)):
        if done == True:                                        # Ends loop once the QA item is found
            break
        elif trans[trans_num] == 'Question-and-Answer Session':
            pree_QA_title   = trans[:trans_num - 1]
            post_QA_title   = trans[trans_num:]
            done            = True
        else:
            pass
    df['speech'][count] = pree_QA_title
    df['QA'][count]     = post_QA_title
    count += 1

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


TypeError: object of type 'int' has no len()

## Temp DF that failed

In [703]:
# Only for the QA as a fix
count = 0
for trans in url_df['transcript']:
    done = False
    for trans_num in range(len(trans)):
        if done == True:
            break
        elif trans[trans_num] == 'Question-and-Answer Session':
            post_QA_title   = trans[trans_num:]
            done            = True
        else:
            pass
    print(count, 'Done')
    temp_df['QA'][count]     = post_QA_title
    count += 1


temp_df['QA']   = temp_df['QA'].map(lambda x: ''.join([piece for piece in x if len(piece) >= 25]))

0 Done
1 Done
2 Done
3 Done
4 Done


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


5 Done
6 Done
7 Done
8 Done
9 Done
10 Done
11 Done
12 Done
13 Done
14 Done
15 Done
16 Done
17 Done
18 Done
19 Done
20 Done
21 Done
22 Done
23 Done
24 Done
25 Done
26 Done
27 Done
28 Done
29 Done
30 Done
31 Done
32 Done
33 Done
34 Done
35 Done
36 Done
37 Done
38 Done
39 Done
40 Done
41 Done
42 Done
43 Done
44 Done
45 Done
46 Done
47 Done
48 Done
49 Done
50 Done
51 Done
52 Done
53 Done
54 Done
55 Done
56 Done
57 Done
58 Done
59 Done
60 Done
61 Done
62 Done
63 Done
64 Done
65 Done
66 Done
67 Done
68 Done
69 Done
70 Done
71 Done
72 Done
73 Done
74 Done
75 Done
76 Done
77 Done
78 Done
79 Done
80 Done
81 Done
82 Done
83 Done
84 Done
85 Done
86 Done
87 Done
88 Done
89 Done
90 Done
91 Done
92 Done
93 Done
94 Done
95 Done
96 Done
97 Done
98 Done
99 Done
100 Done
101 Done
102 Done
103 Done
104 Done
105 Done
106 Done
107 Done
108 Done
109 Done
110 Done
111 Done
112 Done
113 Done
114 Done
115 Done
116 Done
117 Done
118 Done
119 Done
120 Done
121 Done
122 Done
123 Done
124 Done
125 Done
126 Done
12

932 Done
933 Done
934 Done
935 Done
936 Done
937 Done
938 Done
939 Done
940 Done
941 Done
942 Done
943 Done
944 Done
945 Done
946 Done
947 Done
948 Done
949 Done
950 Done
951 Done
952 Done
953 Done
954 Done
955 Done
956 Done
957 Done
958 Done
959 Done
960 Done
961 Done
962 Done
963 Done
964 Done
965 Done
966 Done
967 Done
968 Done
969 Done
970 Done
971 Done
972 Done
973 Done
974 Done
975 Done
976 Done
977 Done
978 Done
979 Done
980 Done
981 Done
982 Done
983 Done
984 Done
985 Done
986 Done
987 Done
988 Done
989 Done
990 Done
991 Done
992 Done
993 Done
994 Done
995 Done
996 Done
997 Done
998 Done
999 Done
1000 Done
1001 Done
1002 Done
1003 Done
1004 Done
1005 Done
1006 Done
1007 Done
1008 Done
1009 Done
1010 Done
1011 Done
1012 Done
1013 Done
1014 Done
1015 Done
1016 Done
1017 Done
1018 Done
1019 Done
1020 Done
1021 Done
1022 Done
1023 Done
1024 Done
1025 Done
1026 Done
1027 Done
1028 Done
1029 Done
1030 Done
1031 Done
1032 Done
1033 Done
1034 Done
1035 Done
1036 Done
1037 Done
1038 Don

1764 Done
1765 Done
1766 Done
1767 Done
1768 Done
1769 Done
1770 Done
1771 Done
1772 Done
1773 Done
1774 Done
1775 Done
1776 Done
1777 Done
1778 Done
1779 Done
1780 Done
1781 Done
1782 Done
1783 Done
1784 Done
1785 Done
1786 Done
1787 Done
1788 Done
1789 Done
1790 Done
1791 Done
1792 Done
1793 Done
1794 Done
1795 Done
1796 Done
1797 Done
1798 Done
1799 Done
1800 Done
1801 Done
1802 Done
1803 Done
1804 Done
1805 Done
1806 Done
1807 Done
1808 Done
1809 Done
1810 Done
1811 Done
1812 Done
1813 Done
1814 Done
1815 Done
1816 Done
1817 Done
1818 Done
1819 Done
1820 Done
1821 Done
1822 Done
1823 Done
1824 Done
1825 Done
1826 Done
1827 Done
1828 Done
1829 Done
1830 Done
1831 Done
1832 Done
1833 Done
1834 Done
1835 Done
1836 Done
1837 Done
1838 Done
1839 Done
1840 Done
1841 Done
1842 Done
1843 Done
1844 Done
1845 Done
1846 Done
1847 Done
1848 Done
1849 Done
1850 Done
1851 Done
1852 Done
1853 Done
1854 Done
1855 Done
1856 Done
1857 Done
1858 Done
1859 Done
1860 Done
1861 Done
1862 Done
1863 Done


2585 Done
2586 Done
2587 Done
2588 Done
2589 Done
2590 Done
2591 Done
2592 Done
2593 Done
2594 Done
2595 Done
2596 Done
2597 Done
2598 Done
2599 Done
2600 Done
2601 Done
2602 Done
2603 Done
2604 Done
2605 Done
2606 Done
2607 Done
2608 Done
2609 Done
2610 Done
2611 Done
2612 Done
2613 Done
2614 Done
2615 Done
2616 Done
2617 Done
2618 Done
2619 Done
2620 Done
2621 Done
2622 Done
2623 Done
2624 Done
2625 Done
2626 Done
2627 Done
2628 Done
2629 Done
2630 Done
2631 Done
2632 Done
2633 Done
2634 Done
2635 Done
2636 Done
2637 Done
2638 Done
2639 Done
2640 Done
2641 Done
2642 Done
2643 Done
2644 Done
2645 Done
2646 Done
2647 Done
2648 Done
2649 Done
2650 Done
2651 Done
2652 Done
2653 Done
2654 Done
2655 Done
2656 Done
2657 Done
2658 Done
2659 Done
2660 Done
2661 Done
2662 Done
2663 Done
2664 Done
2665 Done
2666 Done
2667 Done
2668 Done
2669 Done
2670 Done
2671 Done
2672 Done
2673 Done
2674 Done
2675 Done
2676 Done
2677 Done
2678 Done
2679 Done
2680 Done
2681 Done
2682 Done
2683 Done
2684 Done


3406 Done
3407 Done
3408 Done
3409 Done
3410 Done
3411 Done
3412 Done
3413 Done
3414 Done
3415 Done
3416 Done
3417 Done
3418 Done
3419 Done
3420 Done
3421 Done
3422 Done
3423 Done
3424 Done
3425 Done
3426 Done
3427 Done
3428 Done
3429 Done
3430 Done
3431 Done
3432 Done
3433 Done
3434 Done
3435 Done
3436 Done
3437 Done
3438 Done
3439 Done
3440 Done
3441 Done
3442 Done
3443 Done
3444 Done
3445 Done
3446 Done
3447 Done
3448 Done
3449 Done
3450 Done
3451 Done
3452 Done
3453 Done
3454 Done
3455 Done
3456 Done
3457 Done
3458 Done
3459 Done
3460 Done
3461 Done
3462 Done
3463 Done
3464 Done
3465 Done
3466 Done
3467 Done
3468 Done
3469 Done
3470 Done
3471 Done
3472 Done
3473 Done
3474 Done
3475 Done
3476 Done
3477 Done
3478 Done
3479 Done
3480 Done
3481 Done
3482 Done
3483 Done
3484 Done
3485 Done
3486 Done
3487 Done
3488 Done
3489 Done
3490 Done
3491 Done
3492 Done
3493 Done
3494 Done
3495 Done
3496 Done
3497 Done
3498 Done
3499 Done
3500 Done
3501 Done
3502 Done
3503 Done
3504 Done
3505 Done


TypeError: object of type 'int' has no len()

In [705]:
temp_df['QA']   = url_df['QA'].map(lambda x: ''.join([piece for piece in x if len(piece) >= 25][]))

TypeError: 'float' object is not iterable

In [684]:
temp_df = url_df.head(3500)

In [697]:
temp_df.head()
for row_num in range(len(temp_df['speech'])):
    temp_df['speech'][row_num] = ''.join([piece for piece in temp_df['speech'][row_num] if len(piece) >= 35][1:])
    temp_df['QA'][row_num]     = ''.join([piece for piece in temp_df['speech'][row_num] if len(piece) >= 25])
    print(row_num, 'Done')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2961: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing t

0 Done
1 Done
2 Done
3 Done
4 Done
5 Done
6 Done
7 Done
8 Done
9 Done
10 Done
11 Done
12 Done
13 Done
14 Done
15 Done
16 Done
17 Done
18 Done
19 Done
20 Done
21 Done
22 Done
23 Done
24 Done
25 Done
26 Done
27 Done
28 Done
29 Done
30 Done
31 Done
32 Done
33 Done
34 Done
35 Done
36 Done
37 Done
38 Done
39 Done
40 Done
41 Done
42 Done
43 Done
44 Done
45 Done
46 Done
47 Done
48 Done
49 Done
50 Done
51 Done
52 Done
53 Done
54 Done
55 Done
56 Done
57 Done
58 Done
59 Done
60 Done
61 Done
62 Done
63 Done
64 Done
65 Done
66 Done
67 Done
68 Done
69 Done
70 Done
71 Done
72 Done
73 Done
74 Done
75 Done
76 Done
77 Done
78 Done
79 Done
80 Done
81 Done
82 Done
83 Done
84 Done
85 Done
86 Done
87 Done
88 Done
89 Done
90 Done
91 Done
92 Done
93 Done
94 Done
95 Done
96 Done
97 Done
98 Done
99 Done
100 Done
101 Done
102 Done
103 Done
104 Done
105 Done
106 Done
107 Done
108 Done
109 Done
110 Done
111 Done
112 Done
113 Done
114 Done
115 Done
116 Done
117 Done
118 Done
119 Done
120 Done
121 Done
122 Done
123

924 Done
925 Done
926 Done
927 Done
928 Done
929 Done
930 Done
931 Done
932 Done
933 Done
934 Done
935 Done
936 Done
937 Done
938 Done
939 Done
940 Done
941 Done
942 Done
943 Done
944 Done
945 Done
946 Done
947 Done
948 Done
949 Done
950 Done
951 Done
952 Done
953 Done
954 Done
955 Done
956 Done
957 Done
958 Done
959 Done
960 Done
961 Done
962 Done
963 Done
964 Done
965 Done
966 Done
967 Done
968 Done
969 Done
970 Done
971 Done
972 Done
973 Done
974 Done
975 Done
976 Done
977 Done
978 Done
979 Done
980 Done
981 Done
982 Done
983 Done
984 Done
985 Done
986 Done
987 Done
988 Done
989 Done
990 Done
991 Done
992 Done
993 Done
994 Done
995 Done
996 Done
997 Done
998 Done
999 Done
1000 Done
1001 Done
1002 Done
1003 Done
1004 Done
1005 Done
1006 Done
1007 Done
1008 Done
1009 Done
1010 Done
1011 Done
1012 Done
1013 Done
1014 Done
1015 Done
1016 Done
1017 Done
1018 Done
1019 Done
1020 Done
1021 Done
1022 Done
1023 Done
1024 Done
1025 Done
1026 Done
1027 Done
1028 Done
1029 Done
1030 Done
1031 D

1753 Done
1754 Done
1755 Done
1756 Done
1757 Done
1758 Done
1759 Done
1760 Done
1761 Done
1762 Done
1763 Done
1764 Done
1765 Done
1766 Done
1767 Done
1768 Done
1769 Done
1770 Done
1771 Done
1772 Done
1773 Done
1774 Done
1775 Done
1776 Done
1777 Done
1778 Done
1779 Done
1780 Done
1781 Done
1782 Done
1783 Done
1784 Done
1785 Done
1786 Done
1787 Done
1788 Done
1789 Done
1790 Done
1791 Done
1792 Done
1793 Done
1794 Done
1795 Done
1796 Done
1797 Done
1798 Done
1799 Done
1800 Done
1801 Done
1802 Done
1803 Done
1804 Done
1805 Done
1806 Done
1807 Done
1808 Done
1809 Done
1810 Done
1811 Done
1812 Done
1813 Done
1814 Done
1815 Done
1816 Done
1817 Done
1818 Done
1819 Done
1820 Done
1821 Done
1822 Done
1823 Done
1824 Done
1825 Done
1826 Done
1827 Done
1828 Done
1829 Done
1830 Done
1831 Done
1832 Done
1833 Done
1834 Done
1835 Done
1836 Done
1837 Done
1838 Done
1839 Done
1840 Done
1841 Done
1842 Done
1843 Done
1844 Done
1845 Done
1846 Done
1847 Done
1848 Done
1849 Done
1850 Done
1851 Done
1852 Done


2574 Done
2575 Done
2576 Done
2577 Done
2578 Done
2579 Done
2580 Done
2581 Done
2582 Done
2583 Done
2584 Done
2585 Done
2586 Done
2587 Done
2588 Done
2589 Done
2590 Done
2591 Done
2592 Done
2593 Done
2594 Done
2595 Done
2596 Done
2597 Done
2598 Done
2599 Done
2600 Done
2601 Done
2602 Done
2603 Done
2604 Done
2605 Done
2606 Done
2607 Done
2608 Done
2609 Done
2610 Done
2611 Done
2612 Done
2613 Done
2614 Done
2615 Done
2616 Done
2617 Done
2618 Done
2619 Done
2620 Done
2621 Done
2622 Done
2623 Done
2624 Done
2625 Done
2626 Done
2627 Done
2628 Done
2629 Done
2630 Done
2631 Done
2632 Done
2633 Done
2634 Done
2635 Done
2636 Done
2637 Done
2638 Done
2639 Done
2640 Done
2641 Done
2642 Done
2643 Done
2644 Done
2645 Done
2646 Done
2647 Done
2648 Done
2649 Done
2650 Done
2651 Done
2652 Done
2653 Done
2654 Done
2655 Done
2656 Done
2657 Done
2658 Done
2659 Done
2660 Done
2661 Done
2662 Done
2663 Done
2664 Done
2665 Done
2666 Done
2667 Done
2668 Done
2669 Done
2670 Done
2671 Done
2672 Done
2673 Done


3395 Done
3396 Done
3397 Done
3398 Done
3399 Done
3400 Done
3401 Done
3402 Done
3403 Done
3404 Done
3405 Done
3406 Done
3407 Done
3408 Done
3409 Done
3410 Done
3411 Done
3412 Done
3413 Done
3414 Done
3415 Done
3416 Done
3417 Done
3418 Done
3419 Done
3420 Done
3421 Done
3422 Done
3423 Done
3424 Done
3425 Done
3426 Done
3427 Done
3428 Done
3429 Done
3430 Done
3431 Done
3432 Done
3433 Done
3434 Done
3435 Done
3436 Done
3437 Done
3438 Done
3439 Done
3440 Done
3441 Done
3442 Done
3443 Done
3444 Done
3445 Done
3446 Done
3447 Done
3448 Done
3449 Done
3450 Done
3451 Done
3452 Done
3453 Done
3454 Done
3455 Done
3456 Done
3457 Done
3458 Done
3459 Done
3460 Done
3461 Done
3462 Done
3463 Done
3464 Done
3465 Done
3466 Done
3467 Done
3468 Done
3469 Done
3470 Done
3471 Done
3472 Done
3473 Done
3474 Done
3475 Done
3476 Done
3477 Done
3478 Done
3479 Done
3480 Done
3481 Done
3482 Done
3483 Done
3484 Done
3485 Done
3486 Done
3487 Done
3488 Done
3489 Done
3490 Done
3491 Done
3492 Done
3493 Done
3494 Done


In [701]:
temp_df['QA'][2]

''

In [ ]:
temp_df['speech_sentiment']  = temp_df['speech'].map(lambda x: Sentamentize(x))
print('Speech Sentiment Done')
temp_df['speech_complexity'] = temp_df['speech'].map(lambda x: textstat.gunning_fog(x))
print('Speech Complexity Done')
temp_df['QA_sentiment']      = temp_df['QA'].map(lambda x: Sentamentize(x))
print('QA Sentiment Done')
temp_df['QA_complexity']     = temp_df['QA'].map(lambda x: textstat.gunning_fog(x))
print('QA Complexity Done')

temp_df.head()

In [ ]:
temp_df['speech'] = temp_df[''].map(lambda x: ''.join([piece for piece in x if len(piece) >= 35][1:])
temp_df['QA']     = temp_df[''].map(lambda x: ''.join([piece for piece in x if len(piece) >= 25])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2961: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing t

0 Done
1 Done
2 Done
3 Done
4 Done
5 Done
6 Done
7 Done
8 Done
9 Done
10 Done
11 Done
12 Done
13 Done
14 Done
15 Done
16 Done
17 Done
18 Done
19 Done
20 Done
21 Done
22 Done
23 Done
24 Done
25 Done
26 Done
27 Done
28 Done
29 Done
30 Done
31 Done
32 Done
33 Done
34 Done
35 Done
36 Done
37 Done
38 Done
39 Done
40 Done
41 Done
42 Done
43 Done
44 Done
45 Done
46 Done
47 Done
48 Done
49 Done
50 Done
51 Done
52 Done
53 Done
54 Done
55 Done
56 Done
57 Done
58 Done
59 Done
60 Done
61 Done
62 Done
63 Done
64 Done
65 Done
66 Done
67 Done
68 Done
69 Done
70 Done
71 Done
72 Done
73 Done
74 Done
75 Done
76 Done
77 Done
78 Done
79 Done
80 Done
81 Done
82 Done
83 Done
84 Done
85 Done
86 Done
87 Done
88 Done
89 Done
90 Done
91 Done
92 Done
93 Done
94 Done
95 Done
96 Done
97 Done
98 Done
99 Done
100 Done
101 Done
102 Done
103 Done
104 Done
105 Done
106 Done
107 Done
108 Done
109 Done
110 Done
111 Done
112 Done
113 Done
114 Done
115 Done
116 Done
117 Done
118 Done
119 Done
120 Done
121 Done
122 Done
123

924 Done
925 Done
926 Done
927 Done
928 Done
929 Done
930 Done
931 Done
932 Done
933 Done
934 Done
935 Done
936 Done
937 Done
938 Done
939 Done
940 Done
941 Done
942 Done
943 Done
944 Done
945 Done
946 Done
947 Done
948 Done
949 Done
950 Done
951 Done
952 Done
953 Done
954 Done
955 Done
956 Done
957 Done
958 Done
959 Done
960 Done
961 Done
962 Done
963 Done
964 Done
965 Done
966 Done
967 Done
968 Done
969 Done
970 Done
971 Done
972 Done
973 Done
974 Done
975 Done
976 Done
977 Done
978 Done
979 Done
980 Done
981 Done
982 Done
983 Done
984 Done
985 Done
986 Done
987 Done
988 Done
989 Done
990 Done
991 Done
992 Done
993 Done
994 Done
995 Done
996 Done
997 Done
998 Done
999 Done
1000 Done
1001 Done
1002 Done
1003 Done
1004 Done
1005 Done
1006 Done
1007 Done
1008 Done
1009 Done
1010 Done
1011 Done
1012 Done
1013 Done
1014 Done
1015 Done
1016 Done
1017 Done
1018 Done
1019 Done
1020 Done
1021 Done
1022 Done
1023 Done
1024 Done
1025 Done
1026 Done
1027 Done
1028 Done
1029 Done
1030 Done
1031 D

1753 Done
1754 Done
1755 Done
1756 Done
1757 Done
1758 Done
1759 Done
1760 Done
1761 Done
1762 Done
1763 Done
1764 Done
1765 Done
1766 Done
1767 Done
1768 Done
1769 Done
1770 Done
1771 Done
1772 Done
1773 Done
1774 Done
1775 Done
1776 Done
1777 Done
1778 Done
1779 Done
1780 Done
1781 Done
1782 Done
1783 Done
1784 Done
1785 Done
1786 Done
1787 Done
1788 Done
1789 Done
1790 Done
1791 Done
1792 Done
1793 Done
1794 Done
1795 Done
1796 Done
1797 Done
1798 Done
1799 Done
1800 Done
1801 Done
1802 Done
1803 Done
1804 Done
1805 Done
1806 Done
1807 Done
1808 Done
1809 Done
1810 Done
1811 Done
1812 Done
1813 Done
1814 Done
1815 Done
1816 Done
1817 Done
1818 Done
1819 Done
1820 Done
1821 Done
1822 Done
1823 Done
1824 Done
1825 Done
1826 Done
1827 Done
1828 Done
1829 Done
1830 Done
1831 Done
1832 Done
1833 Done
1834 Done
1835 Done
1836 Done
1837 Done
1838 Done
1839 Done
1840 Done
1841 Done
1842 Done
1843 Done
1844 Done
1845 Done
1846 Done
1847 Done
1848 Done
1849 Done
1850 Done
1851 Done
1852 Done


In [ ]:
temp_df['speech_sentiment']  = np.nan
temp_df['speech_complexity'] = np.nan
temp_df['QA_sentiment']      = np.nan
temp_df['QA_complexity']     = np.nan

In [ ]:
temp_df.head()
for row_num in range(len(temp_df['speech'])):
    temp_df['speech_sentiment'][row_num]  = Sentamentize(speech)
    temp_df['speech_complexity'][row_num] = np.nan
    temp_df['QA_sentiment'][row_num]      = Sentamentize(speech)
    temp_df['QA_complexity'][row_num]     = np.nan
#    temp_df['speech'][row_num] = ''.join([piece for piece in temp_df['speech'][row_num] if len(piece) >= 35][1:])
#    temp_df['QA'][row_num]     = ''.join([piece for piece in temp_df['speech'][row_num] if len(piece) >= 25])
    print(row_num, 'Done')
    
Sentamentize(speech),
            'speech_complexity': textstat.gunning_fog(speech),
            'QA_sentiment'     : Sentamentize(QA),
            'QA_complexity'    : textstat.gunning_fog(QA)

In [509]:
browser         = open_browser()
num_transcripts = 30

    for ts_num in range(1,num_transcripts):
        p_elements              = [item.text for item in article.find_all('p')]
        title                   = p_elements[0]
        print('scraping ',title)
        sleep()
        
        # Finding the seperation between main speech(es) and QA section
        done = False
        for item_num in range(len(p_elements)):
            if done == True:
                break
            elif p_elements[item_num] == 'Question-and-Answer Session':
                pree_QA_title         = p_elements[:item_num - 1]
                post_QA_title         = p_elements[item_num:]
                done                  = True
            else:
                pass
            
        speech = ''.join([i for i in pree_QA_title if len(i) >= 35][1:])
        QA     = ''.join([i for i in post_QA_title if len(i) >= 25])
        current_dict = {
            'title'   : test_title,
            'speech'  : speech,
            'Q_and_A' : QA,
            'speech_sentiment' : Sentamentize(speech),
            'speech_complexity': textstat.gunning_fog(speech),
            'QA_sentiment'     : Sentamentize(QA),
            'QA_complexity'    : textstat.gunning_fog(QA)
        }
        list_of_dicts.append(current_dict)
        
        
df = pd.DataFrame(list_of_dicts, columns = ['title', 'speech', 'Q_and_A',
                                            'speech_sentiment','speech_complexity',
                                            'QA_sentiment','QA_complexity'])

scraping  Klepierre SA (OTCPK:KLPEF) Q4 2018 Earnings Conference Call February  7, 2019  3:00 AM ET
scraping  ESCO Technologies Inc. (NYSE:ESE) Q1 2019 Earnings Conference Call February  7, 2019  5:00 PM ET
scraping  Electrovaya Inc. (OTCQX:EFLVF) Q1 2019 Results Earnings Conference Call February  7, 2019  7:30 AM ET
scraping  Great-West Lifeco’s. (OTCPK:GWLIF) Q4 2018 Results Earnings Conference Call February  7, 2019  3:30 PM ET
scraping  Ultralife Corp. (NASDAQ:ULBI) Q4 2018 Earnings Conference Call February  7, 2019 10:00 AM ET
scraping  Cleveland-Cliffs Inc. (NYSE:CLF) Q4 2018 Earnings Conference Call February  8, 2019  9:00 AM ET
scraping  Kemira Oyj (OTC:KMRAF) Q4 2018 Earnings Conference Call February  8, 2019  3:30 AM ET
scraping  Matrix Service Company (NASDAQ:MTRX) Q2 2019 Earnings Conference Call February  7, 2019 10:30 AM ET


AttributeError: 'NoneType' object has no attribute 'find_all'

# Everything else

In [528]:
test_elements = browser.find_elements_by_class_name('dashboard-article-link')

In [716]:
url_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6180 entries, 0 to 6179
Data columns (total 6 columns):
header        6180 non-null object
page_list     5850 non-null float64
url           6180 non-null object
speech        3651 non-null object
QA            3651 non-null object
transcript    6180 non-null object
dtypes: float64(1), object(5)
memory usage: 289.8+ KB


In [714]:
#from time import time
#from datetime import datetime
url_df.to_csv('./../url_df.csv', index= False)

In [636]:
#browser = open_browser(str(random.randrange(0,1000)) + "Style & Grace")
#browser.get(url_df['url'][row_num])
test_res = requests.get(url_df['url'][row_num])

In [485]:
speech  = ''.join(pree_QA_title)
QA      = ''.join(post_QA_title)
test_list_of_dicts = [
{
    'title'   : test_title,
    'speech'  : speech,
    'Q_and_A' : QA,
    'speech_sentiment' : Sentamentize(speech),
    'speech_complexity': textstat.gunning_fog(speech),
    'QA_sentiment'     : Sentamentize(QA),
    'QA_complexity'    : textstat.gunning_fog(QA)
}]

In [486]:
pd.DataFrame(test_list_of_dicts, columns = ['title', 'speech', 'Q_and_A',
                                            'speech_sentiment','speech_complexity',
                                            'QA_sentiment','QA_complexity'])

,title,speech,Q_and_A,speech_sentiment,speech_complexity,QA_sentiment,QA_complexity
0,BrightView Holdings (NYSE:BV) Q1 2019 Earnings...,"Good morning, and welcome to BrightView's Fisc...",Thank you. Your first question comes from Geor...,0.146584,19.4,0.131598,11.38


## Code Recycling Bin
- For when I'm not yet ready to delete certain old code.

In [285]:
# Code used for testing which technique was the more accurate.
l = []
for i in range(0,11):
    l.append([i for i in range(0,11)]+ ['hurray happy great awesome but unfortunatly I hate it'])
test_df = pd.DataFrame(l)

def math(x):
    x = (x - 2)*22
    return x

test_df['old_tech']     = test_df[5].map(lambda x: math(x))
#test_df['complex_tech'] = test_df.apply(lambda x: pd.Series(math(x),
#                                    index=[5]), axis=1)
test_df['sentiment_s'] = test_df[11].map(lambda x: Sentamentize(x))
test_df['sentiment_l'] = test_df.apply(lambda x: pd.Series(Sentamentize(x),
                                       index=[11]), axis=1)
test_df
# I guess, yeah, stick with the short version then

In [425]:
current_item = ''
count = 0
while current_item != 'Operator':
    current_item = test_ps[count]
    count += 1
    print(count, current_item)
    #test_ps.remove(test_ps[count])
test_ps = test_ps[count-1:]

1 BrightView Holdings (NYSE:BV) Q1 2019 Earnings Conference Call February  7, 2019 10:00 AM ET
2 Company Participants
3 Dan Schleiniger - VP, IR
4 Andrew Masterman - CEO
5 John Feenan - CFO
6 Conference Call Participants
7 George Tong - Goldman Sachs
8 Judah Sokel - JPMorgan
9 Andrew Wittmann - Baird
10 Tim Mulrooney - William Blair
11 Phil Ng - Jefferies
12 Shlomo Rosenbaum - Stifel
13 Hamzah Mazari - Macquarie
14 Deane Dray - RBC Capital Markets
15 Dan Dolev - Nomura
16 Kevin McVeigh - Credit Suisse
17 Operator


In [ ]:
def Complexity(x):
    return textstat.gunning_fog(x)

In [502]:
test_list = [1,2,3,4,5,6,7,8,9]
count = 0
#current_item = ''
done = False
for i in range(4):
    print(i)
    for c_item in test_list:
        if done == True:
            print('done')
            break
        elif c_item == 6:
            list_1 = test_list[:c_item - 1]
            list_2 = test_list[c_item:]
            done = True
            print('found')
        else:
            pass
print(list_1,list_2)
    


0
found
done
1
done
2
done
3
done
[1, 2, 3, 4, 5] [7, 8, 9]


In [ ]:
    
#    'Question-and-Answer Session'
#while current_item != 'Question-and-Answer Session':
#    try:
#        current_item = test_list[count]
#        count += 1
#        print('huh')
#    except IndexError:
#        print('cool')
#        break
#print('blep')

In [484]:
current_item = ''
count = 0
while current_item != 'Question-and-Answer Session':
    try:
        current_item = test_ps[count]
        count += 1
    except IndexError:
        break
    #print(count, current_item)
pree_QA_title = [item for item in test_ps[:count - 2] if len(item) >= 35][1:]
post_QA_title = [item for item in test_ps[count:]     if len(item) >= 25] 

In [538]:
# Original version for scraping the page lists for transcript urls
# Check if it's been scraped yet or not
# Get full list of all individual page urls that I want to scrape
browser = open_browser()
for page_num in range(4,7):
    current_ts_list = 'https://seekingalpha.com/earnings/earnings-call-transcripts/' + str(page_num)
    browser.get(current_ts_list)
    elements_list = browser.find_elements_by_class_name('dashboard-article-link')
    urls    = [el.get_attribute('href') + '?part=single' for el in elements_list]
    headers = [el.text for el in elements_list]
    print('scraping: Page',page_num)
    sleep(20)
    for transcript_num in range(len(urls)):
        current_transcript = {
            'header' : headers[transcript_num],
            'url'    : urls[transcript_num]
        }
        url_list.append(current_transcript)

scraping: Page 4
scraping: Page 5
scraping: Page 6


In [712]:
for i in range(20):
    try:
        for row_num in range(len(url_df['transcript'])):
            if url_df['transcript'][row_num] == 0:
                # Go to url and obtain the full transcript
                browser = open_browser()#str(random.randrange(0,1000)) + "Style & Grace")
                browser.get(url_df['url'][row_num])
                print('scraping',row_num,
                      datetime.fromtimestamp(time()).strftime('%H:%M:%S'),
                      url_df['header'][row_num][:85])
                
                soup        = BeautifulSoup(browser.page_source)
                article     = soup.find('article')
                url_df['transcript'][row_num]  = [item.text for item in article.find_all('p')]
                browser.close()
                sleep(20)
            else:
                pass
    except:
        print('--------')
        print('ERROR', i)
        print('--------')
        sleep(100)
        browser.close()

scraping 3651 14:51:24 Mallinckrodt Plc (MNK) Q3 2018 Results - Earnings Call Transcript


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


scraping 3652 14:52:00 j2 Global (JCOM) Q3 2018 Results - Earnings Call Transcript
scraping 3653 14:52:36 Jones Lang LaSalle (JLL) Q3 2018 Results - Earnings Call Transcript
scraping 3654 14:53:10 Taro Pharmaceutical Industries Ltd (TARO) CEO Uday Baldota on 2018/19 First Half Resu
scraping 3655 14:53:42 Franco-Nevada Corp (FNV) on Q3 2018 Results - Earnings Call Transcript
scraping 3656 14:54:14 Ciner Resources' (CINR) CEO Kirk Milling on Q3 2018 Results - Earnings Call Transcrip
scraping 3657 14:54:43 NIO Inc. (NIO) CEO William Li on Q3 2018 Results - Earnings Call Transcript
scraping 3658 14:55:18 BioCryst's (BCRX) CEO Jon Stonehouse on Q3 2018 Results - Earnings Call Transcript
scraping 3659 14:55:50 CVS Health (CVS) Q3 2018 Results - Earnings Call Transcript
scraping 3660 14:56:25 Ocwen Financial Corporation (OCN) CEO Glen Messina on Q3 2018 Results - Earnings Call
scraping 3661 14:57:04 OnDeck Capital's (ONDK) CEO Noah Breslow on Q3 2018 Results - Earnings Call Transcrip
scraping

scraping 3731 15:37:35 Astronics Corporation (ATRO) CEO Pete Gundermann on Q3 2018 Results - Earnings Call T
scraping 3732 15:38:20 Kemper Corporation (KMPR) CEO Joe Lacher on Q3 2018 Results - Earnings Call Transcrip
scraping 3733 15:38:50 Mylan's (MYL) CEO Heather Bresch on Q3 2018 Results - Earnings Call Transcript
scraping 3734 15:39:22 Xencor, Inc. (XNCR) CEO Bassil Dahiyat on Q3 2018 Results - Earnings Call Transcript
scraping 3735 15:39:55 OptimizeRx Corp. (OPRX) CEO William Febbo on Q3 2018 Results - Earnings Call Transcri
scraping 3736 15:40:32 Hemisphere Media Group's (HMTV) CEO Alan Sokol on Q3 2018 Results - Earnings Call Tra
scraping 3737 15:41:06 Calix Inc. (CALX) CEO Carl Russo on Q3 2018 Results - Earnings Call Transcript
scraping 3738 15:41:41 Dufry AG (DUFRY) CEO Julián González on Q3 2018 Results - Earnings Call Transcript
scraping 3739 15:42:12 Fiesta Restaurant Group's (FRGI) CEO Rich Stockinger on Q3 2018 Results - Earnings Ca
scraping 3740 15:42:47 Akcea Therapeu

scraping 3807 16:28:43 TGS Nopec Geophysical Company (TGSNF) CEO Kristian Johansen on Q3 2018 Results - Earn
scraping 3808 16:29:17 Illumina, Inc. (ILMN) CEO Francis deSouza Hosts Acquisition of Pacific Biosciences Co
scraping 3809 16:29:54 KDDI Corp.'s (KDDIF) Management on Fiscal Q2 2019 Results - Earnings Call Transcript
scraping 3810 16:30:31 Rogers Corporation (ROG) CEO Bruce Hoechner on Q3 2018 Results - Earnings Call Transc
scraping 3811 16:31:07 Takeda Pharmaceutical Co. Ltd. (TKPHF) CEO Christophe Weber on Q2 2019 Results - Earn
--------
ERROR 1
--------
scraping 3812 16:38:22 Cross Country Healthcare, Inc. (CCRN) CEO Bill Grubbs on Q3 2018 Results - Earnings C
scraping 3813 16:38:59 Westell Technologies, Inc. (WSTL) CEO Stephen John on Q2 2019 Results - Earnings Call
scraping 3814 16:39:31 NOW Inc. (DNOW) CEO Robert Workman on Q3 2018 Results - Earnings Call Transcript
scraping 3815 16:40:08 AMG Advanced Metallurgical Group NV (AMVMF) CEO Dr. Heinz Schimmelbusch on Q3 2018 Re

scraping 3883 17:26:57 Banco Bradesco, S.A. (BBD) CEO Octavio de Lazari, Jr. on Q3 2018 Results - Earnings C
scraping 3884 17:27:34 FreightCar America, Inc. (RAIL) CEO Jim Meyer on Q3 2018 Results - Earnings Call Tran
scraping 3885 17:28:06 Grupo Financiero Santander S.A.B. de C.V. (BSMX) CEO Hector Grisi on Q3 2018 Results 
scraping 3886 17:28:35 Hilton Grand Vacations Inc. (HGV) CEO Mark Wang on Q3 2018 Results - Earnings Call Tr
scraping 3887 17:29:17 Heritage Insurance Holdings, Inc (HRTG) CEO Bruce Lucas on Q3 2018 Results - Earnings
scraping 3888 17:29:53 Nanophase Technologies Corp. (NANX) CEO Jess Jankowski on Q3 2018 Results - Earnings 
scraping 3889 17:30:29 Jones Energy, Inc. (JONE) CEO Carl Giesler on Q3 2018 Results - Earnings Call Transcr
scraping 3890 17:31:06 Wesco International, Inc. (WCC) CEO John Engel on Q3 2018 Results - Earnings Call Tra
scraping 3891 17:31:42 Insperity, Inc. (NSP) CEO Paul Sarvadi on Q3 2018 Results - Earnings Call Transcript
scraping 3892 17:32:

scraping 3959 18:12:37 Endologix Inc. (ELGX) CEO John Onopchenko on Q3 2018 Results - Earnings Call Transcri
scraping 3960 18:13:19 CONMED Corporation (CNMD) CEO Curt Hartman on Q3 2018 Results - Earnings Call Transcr
scraping 3961 18:13:53 Douglas Emmett, Inc. (DEI) CEO Jordan Kaplan on Q3 2018 Results - Earnings Call Trans
scraping 3962 18:14:30 Ashford Hospitality Trust Inc. (AHT) CEO Douglas Kessler on Q3 2018 Results - Earning
scraping 3963 18:15:05 M.D.C. Holdings' (MDC) CEO Larry Mizel on Q3 2018 Results - Earnings Call Transcript
scraping 3964 18:15:37 The Ensign Group's (ENSG) CEO Christopher Christensen on Q3 2018 Results - Earnings C
scraping 3965 18:16:13 Shell Midstream Partners, LP (SHLX) CEO Ulf Quellmann on Q3 2018 Results - Earnings C
scraping 3966 18:16:48 Imperial Oil Ltd (IMO) CEO Richard Kruger on Q3 2018 Results - Earnings Call Transcri
scraping 3967 18:17:23 Hudson Pacific Properties' (HPP) CEO Victor Coleman on Q3 2018 Results - Earnings Cal
scraping 3968 18:18:

WebDriverException: Message: invalid session id
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.14.0 x86_64)


In [456]:
test_ps    = [item.text for item in test_article.find_all('p')]
test_title = test_ps[0]
Test_CEO   = [item for item in test_ps if "CEO" in item][0]
Test_CFO   = [item for item in test_ps if "CFO" in item][0]